In [1]:
from DELCgen import *
import scipy.stats as st
import matplotlib.pyplot as plt 
from constants import *
from scipy.interpolate import interp1d
import naima
import astropy.units as u
import subroutines as sub
import os
import simulation as sim
from tqdm import tqdm

This piece of code runs a simple simulation where a CR population is injected and evolved with a variable jet. We need to define the maximum rigidity which is related to the jet power
$$
R_{\mathrm{max}} = 10^{19}~V\left(\frac{Q_j}{10^{44}~\mathrm{erg~s}^{-1}}\right)^{1/2}
    \left(\frac{\eta}{0.1}\right)
    \left(\frac{u}{0.1c}\right).
$$

the "power threshold" is the inverse expression with $Q_j$ as the subject. We use an input power spectral density (PSD) and jet power probability distribution function (PDF) to generate a time series for the jet power for a given set of PSD and PDF parameters. To do this, we use the method described by Emmanolopoulous et al. and implemented in python by Connolly. The PSD is implemented as a bending power-law model of the form 

$$\mathrm{PSD}(\nu,\bar{Q},\nu_c) = A(\bar{Q}) \frac{\nu^{-\alpha_{\mathrm{low}}}}
               {1+(\nu/\nu_c)^{\alpha_{\mathrm{high}}-\alpha_{\mathrm{low}}}}$$
               
               
We set $\alpha_{\mathrm{low}} = 1$ for a pink noise spectrum, $\alpha_{\mathrm{high}} = 10$ so as to suppress high frequency variability, and $\nu_c = 1$Myr, roughly equal to the light travel time, $\tau_c=L/c$, along a $L=300$kpc jet. The acceleration time relates to $\tau_c$ by $\tau_\mathrm{acc}/\tau_c \approx (Rc)/(uL)$, since the highest energy particles reach Larmor radii comparable to $(u/c)R$ (Hillas criterion). The longest CR acceleration time is therefore shorter than the light travel time along the jet for any fast, long and thin jet ($u/c>R/L$). 

In [2]:
def get_lc(lognorm_params, PSD_params, tbin, Age):
    # Simulation params
    # let's do everything in units of kyr
    # run for 100 Myr (1e5 kyr) in bins of 0.1 Myr
    #lognorm_params = (1.5,0,np.exp(1.5))
    RedNoiseL,RandomSeed,aliasTbin = 100,12,100
    N = Age / tbin

    lc = Simulate_DE_Lightcurve(BendingPL, PSD_params,st.lognorm,lognorm_params,
                                    RedNoiseL=RedNoiseL,aliasTbin=aliasTbin,randomSeed=RandomSeed,LClength=Age, tbin=tbin)

    return (lc)

Now we've set up the basic functions. Let's initialise our parameters for the variable jet history and initialise the tau_loss dictionary which stores the loss times for the different species (calculated using CRPropa).  

In [3]:
# since we're in Kyr units, set up conversion
MYR = 1e3

# set up light curve. pl_index of 1 means red noise.
pl_index = 1

# bend the power law at 1 Myr (1e-3 kyr^-1) and steeply decay it beyond that with index 10. 
# A,v_bend,a_low,a_high,c = 1, 1e-3, pl_index, 10, 1
PSD_params = (1, 1.0 * MYR, pl_index, 10, 1)
tbin = 0.1 * MYR   # 100 kyr 
Age = 100.0 * MYR
Length = int(Age / tbin)

# time is in kyr, so convert to Myr
times = np.arange ( 0, Length*tbin, tbin) 

elems = ["H", "He", "N", "Fe"]
tau_loss = dict()
energies = np.logspace(6,20.5,num=3000)
for i in range(len(elems)):
    tau_loss[elems[i]] = sub.Losses(elems[i])
    tau_loss[elems[i]].interpol(energies)
    
lognorm_params = (1.5,0,np.exp(1.5))
    
# paramaters for lc are lognorm parameters, PSD parameters, tbin and Length (Age is really number of points)
lc = get_lc(lognorm_params, PSD_params, tbin, Length)

Set up arrays to loop over. flux_scales contains the normalisations of our jet power $(\bar{Q})$.
This is actually the median of the distribution, or the mean in log space.
betas is the spectral index of the injected spectrum ($\beta$).

In [4]:
# flux_scales contains the normalisations of our jet power 
# actually the median of the distribution, or the mean in log space.
# betas is the spectral index of the injected spectrum 
flux_scales = np.logspace(43,45,num=10)
betas = np.arange(2,3,0.1)
betas = [2,2.3,2.7]
flux_scales = np.logspace(42.5,44.5,num=10)
print (flux_scales)
sigmas = np.linspace(0.5,3,num=10)

# arrays to store gamma ray and UHECR luminosities
lgammas = np.zeros((len(sigmas), len(flux_scales), len(betas)))
lcrs = np.zeros((len(sigmas), len(flux_scales), len(betas)))

[3.16227766e+42 5.27499706e+42 8.79922544e+42 1.46779927e+43
 2.44843675e+43 4.08423865e+43 6.81292069e+43 1.13646367e+44
 1.89573565e+44 3.16227766e+44]


Let's loop over the flux scales and the spectral indices. The method is as follows. For each value of $\bar{Q}$ and $\beta$, we evolve a CR distribution for each ionic species numerically using the following equation,

$$
    \frac{dn_i(E)}{dt} = S(E/Z_i, f_i, t) - \frac{n(E)}{\tau_\mathrm{esc} (E/Z_i)} - \frac{n(E)}{\tau_\mathrm{loss} (E, Z_i)}.
$$

The total differential CR spectrum is then just $n(E,t)=\sum_i n_i(E,t)f_i$. We follow, e.g., Eichmann et al. 2018 in using a power-law for the injection term $S({\cal R}_i, f_i, t)$ with a sharp cutoff, given by

$$
    S(E/Z_i, f_i, t) = 
    f_i \eta Q_j(t)~\nu_i
    \left(\frac{{\cal R}}{{\cal R}_0}\right)^{-\beta}~f_{\mathrm{cut}}(R_{\mathrm{max}}).
    % f_\mathrm{cut}
$$

where $\nu_i$ is the normalisation of the distribution and is given by $\nu_i=1.0/[\ln({\cal R}_\mathrm{max}(t)/{\cal R}_0]$ for $\beta=2$ and $\nu_i=(2-\beta)/[({\cal R}_\mathrm{max}/{\cal R}_0)^{(2-\beta)}-1]$ for $\beta > 2$.  We set ${\cal R}_\mathrm{max}$ according to the maximum rigidity condition, ${\cal R}_0=1\mathrm{GV}$. 

In [7]:
for i_sigma, SIGMA in enumerate(tqdm(sigmas)):
    
    lognorm_params = (SIGMA,0,np.exp(SIGMA))
    
    # paramaters for lc are lognorm parameters, PSD parameters, tbin and Length (Age is really number of points)
    lc = get_lc(lognorm_params, PSD_params, tbin, Length)
                      
    for i_flux, flux_scale in enumerate(flux_scales):

        # normalise the light curve
        flux = lc.flux * flux_scale 

        # loop over spectral indices
        for i_beta, BETA in enumerate(betas):

            # elemental "abundances" - really injection fractions 
            # this is roughly solar, but should probably be top heavy since it is easier to inject heavy ions, generally
            frac_elem = np.array([1.0,0.1,1e-4,3.16e-05])

            ncr, escaping, lcr = sim.run_jet_simulation(energies, flux_scale, BETA, lc, tau_loss,
                                                        frac_elem=frac_elem, plot_all=False, 
                                                        sigma=SIGMA, R0=1e9, )

            # get approximate gamma ray luminosity around 10 GeV
            select = (energies > 1e10) * (energies < 2e10)
            my_lgamma = np.fabs(np.trapz(energies[select] * EV2ERGS, energies[select]*ncr[0,-1,:][select]))

            # distance to Cen A
            distance = 3.7 * PARSEC * 1e6
            lgammas[i_sigma,i_flux,i_beta] = my_lgamma * 1e-4 * 4.5e-26 * 0.5 * C / 4.0 / PI / distance / distance
            
            select = (energies > 6e19)

            # store the UHECR luminosity 
#             lcr = np.fabs(np.trapz(energies[select] * EV2ERGS, energies[select] * escaping[select]))
            lcrs[i_sigma,i_flux,i_beta] = lcr
            lg = my_lgamma * 1e-4 * 4.5e-26 * 0.5 * C / 4.0 / PI / distance / distance
            
            print ("sigma {:.1f} BETA {:.1f} median lum {:8.4e} mean lum {:8.4e} UHECR LUM: {:8.4e} {:8.4e}"
                   .format(SIGMA, BETA, flux_scale, np.mean(flux), lcr, lg))
                   



  0%|          | 0/10 [00:00<?, ?it/s]

sigma 0.5 BETA 2.0 median lum 3.1623e+42 mean lum 5.8420e+42 UHECR LUM: 2.8694e+37 1.9094e-15
sigma 0.5 BETA 2.3 median lum 3.1623e+42 mean lum 5.8420e+42 UHECR LUM: 2.8277e+35 7.5781e-15
sigma 0.5 BETA 2.7 median lum 3.1623e+42 mean lum 5.8420e+42 UHECR LUM: 1.1036e+32 8.0638e-15
sigma 0.5 BETA 2.0 median lum 5.2750e+42 mean lum 9.7451e+42 UHECR LUM: 5.3455e+37 3.1492e-15
sigma 0.5 BETA 2.3 median lum 5.2750e+42 mean lum 9.7451e+42 UHECR LUM: 5.2013e+35 1.2640e-14
sigma 0.5 BETA 2.7 median lum 5.2750e+42 mean lum 9.7451e+42 UHECR LUM: 1.9802e+32 1.3451e-14
sigma 0.5 BETA 2.0 median lum 8.7992e+42 mean lum 1.6256e+43 UHECR LUM: 9.6029e+37 5.1946e-15
sigma 0.5 BETA 2.3 median lum 8.7992e+42 mean lum 1.6256e+43 UHECR LUM: 9.2764e+35 2.1083e-14
sigma 0.5 BETA 2.7 median lum 8.7992e+42 mean lum 1.6256e+43 UHECR LUM: 3.4657e+32 2.2438e-14
sigma 0.5 BETA 2.0 median lum 1.4678e+43 mean lum 2.7116e+43 UHECR LUM: 1.4220e+38 8.5698e-15
sigma 0.5 BETA 2.3 median lum 1.4678e+43 mean lum 2.7116e+43


 10%|█         | 1/10 [02:34<23:06, 154.03s/it]

sigma 0.5 BETA 2.7 median lum 3.1623e+44 mean lum 5.8420e+44 UHECR LUM: 1.1412e+34 8.0638e-13
sigma 0.8 BETA 2.0 median lum 3.1623e+42 mean lum 9.1111e+42 UHECR LUM: 4.3736e+37 2.9385e-15
sigma 0.8 BETA 2.3 median lum 3.1623e+42 mean lum 9.1111e+42 UHECR LUM: 4.3089e+35 1.1817e-14
sigma 0.8 BETA 2.7 median lum 3.1623e+42 mean lum 9.1111e+42 UHECR LUM: 1.6621e+32 1.2576e-14
sigma 0.8 BETA 2.0 median lum 5.2750e+42 mean lum 1.5198e+43 UHECR LUM: 7.8306e+37 4.8473e-15
sigma 0.8 BETA 2.3 median lum 5.2750e+42 mean lum 1.5198e+43 UHECR LUM: 7.6731e+35 1.9711e-14
sigma 0.8 BETA 2.7 median lum 5.2750e+42 mean lum 1.5198e+43 UHECR LUM: 2.9111e+32 2.0978e-14
sigma 0.8 BETA 2.0 median lum 8.7992e+42 mean lum 2.5352e+43 UHECR LUM: 1.2754e+38 7.9969e-15
sigma 0.8 BETA 2.3 median lum 8.7992e+42 mean lum 2.5352e+43 UHECR LUM: 1.2765e+36 3.2878e-14
sigma 0.8 BETA 2.7 median lum 8.7992e+42 mean lum 2.5352e+43 UHECR LUM: 4.8788e+32 3.4994e-14
sigma 0.8 BETA 2.0 median lum 1.4678e+43 mean lum 4.2290e+43


 20%|██        | 2/10 [05:11<20:40, 155.12s/it]

...

sigma 2.4 BETA 2.7 median lum 3.1623e+44 mean lum 4.5448e+46 UHECR LUM: 7.8670e+35 6.2732e-11
sigma 2.7 BETA 2.0 median lum 3.1623e+42 mean lum 1.0203e+45 UHECR LUM: 3.6367e+39 2.8686e-13
sigma 2.7 BETA 2.3 median lum 3.1623e+42 mean lum 1.0203e+45 UHECR LUM: 4.3269e+37 1.3224e-12
sigma 2.7 BETA 2.7 median lum 3.1623e+42 mean lum 1.0203e+45 UHECR LUM: 1.7342e+34 1.4083e-12
sigma 2.7 BETA 2.0 median lum 5.2750e+42 mean lum 1.7019e+45 UHECR LUM: 6.0066e+39 4.7387e-13
sigma 2.7 BETA 2.3 median lum 5.2750e+42 mean lum 1.7019e+45 UHECR LUM: 7.2171e+37 2.2059e-12
sigma 2.7 BETA 2.7 median lum 5.2750e+42 mean lum 1.7019e+45 UHECR LUM: 2.8929e+34 2.3491e-12
sigma 2.7 BETA 2.0 median lum 8.7992e+42 mean lum 2.8389e+45 UHECR LUM: 9.9200e+39 7.8285e-13
sigma 2.7 BETA 2.3 median lum 8.7992e+42 mean lum 2.8389e+45 UHECR LUM: 1.2037e+38 3.6795e-12
sigma 2.7 BETA 2.7 median lum 8.7992e+42 mean lum 2.8389e+45 UHECR LUM: 4.8253e+34 3.9186e-12
sigma 2.7 BETA 2.0 median lum 1.4678e+43 mean lum 4.7356e+45


 90%|█████████ | 9/10 [17:25<01:39, 99.72s/it] 

sigma 2.7 BETA 2.7 median lum 3.1623e+44 mean lum 1.0203e+47 UHECR LUM: 1.7340e+36 1.4083e-10
sigma 3.0 BETA 2.0 median lum 3.1623e+42 mean lum 2.3657e+45 UHECR LUM: 8.8654e+39 6.5082e-13
sigma 3.0 BETA 2.3 median lum 3.1623e+42 mean lum 2.3657e+45 UHECR LUM: 1.0517e+38 3.0661e-12
sigma 3.0 BETA 2.7 median lum 3.1623e+42 mean lum 2.3657e+45 UHECR LUM: 4.1410e+34 3.2654e-12
sigma 3.0 BETA 2.0 median lum 5.2750e+42 mean lum 3.9462e+45 UHECR LUM: 1.4646e+40 1.0753e-12
sigma 3.0 BETA 2.3 median lum 5.2750e+42 mean lum 3.9462e+45 UHECR LUM: 1.7542e+38 5.1145e-12
sigma 3.0 BETA 2.7 median lum 5.2750e+42 mean lum 3.9462e+45 UHECR LUM: 6.9074e+34 5.4470e-12
sigma 3.0 BETA 2.0 median lum 8.7992e+42 mean lum 6.5827e+45 UHECR LUM: 2.4200e+40 1.7768e-12
sigma 3.0 BETA 2.3 median lum 8.7992e+42 mean lum 6.5827e+45 UHECR LUM: 2.9260e+38 8.5313e-12
sigma 3.0 BETA 2.7 median lum 8.7992e+42 mean lum 6.5827e+45 UHECR LUM: 1.1522e+35 9.0862e-12
sigma 3.0 BETA 2.0 median lum 1.4678e+43 mean lum 1.0981e+46


100%|██████████| 10/10 [18:48<00:00, 112.84s/it][A

sigma 3.0 BETA 2.7 median lum 3.1623e+44 mean lum 2.3657e+47 UHECR LUM: 4.1408e+36 3.2654e-10


Now let's make a movie with ffmpeg (pretty hacky this)

In [ ]:

plt.figure(figsize=(15,4))
plt.suptitle("UHECR Luminosity above 60 EeV")
for i in range(len(betas)):
    plt.subplot(1,3,i+1)
    plt.contourf(sigmas, np.log10(flux_scales), np.log10(lcrs[:,:,i]))
    plt.colorbar()
    plt.contour(sigmas, np.log10(flux_scales), np.log10(lcrs[:,:,i]), levels=(39,), colors=("C3",), linewidths=3)
    plt.title(r"$\beta={:.1f}$".format(betas[i]), fontsize=16)
    plt.xlabel("$\sigma$", fontsize=16)
    if i == 0: 
        plt.ylabel("$\log Q$", fontsize=16)
    #plt.colorbar()
#plt.pcolormesh(sigmas, flux_scales, lcrs[:,:,1])

In [ ]:
plt.figure(figsize=(15,4))
plt.suptitle("Gamma-ray luminosity at 10 GeV")
for i in range(len(betas)):
    plt.subplot(1,3,i+1)
    plt.contourf(sigmas, np.log10(flux_scales), np.log10(lgammas[:,:,i]))
    plt.colorbar()
    plt.contour(sigmas, np.log10(flux_scales), np.log10(lgammas[:,:,i]), colors=("C3",), levels=(-12,), linewidths=3)
    plt.title(r"$\beta={:.1f}$".format(betas[i]), fontsize=16)
    plt.xlabel("$\sigma$", fontsize=16)
    if i == 0: 
        plt.ylabel("$\log Q$", fontsize=16)